In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re

In [ ]:
data = pd.read_csv("brian_evan_data.csv")

In [ ]:
data.shape

In [ ]:
neighborhoods = list(data["Neighborhood"].unique())
len(neighborhoods)

In [ ]:
categorical_features = list(data.dtypes[data.dtypes == object].index)
print(len(categorical_features))
print(categorical_features)

In [ ]:
numerical_features = list(data.dtypes[data.dtypes != object].index)
print(len(numerical_features))
print(numerical_features)

In [ ]:
data = data.drop("price_per_area", axis = 1)

In [ ]:
numerical_features = list(data.dtypes[data.dtypes != object].index)
print(len(numerical_features))
print(numerical_features)

In [ ]:
# R_data = pd.DataFrame(data = [data["SalePrice"], data["1stFlrSF"], data["2ndFlrSF"], data["Fireplaces"], 
#                               data["TotalBsmtSF"], data["ExterQual"], data["HeatingQC"],
#                               data["HalfBath"], data["BsmtQual"], data["YearRemodAdd"], data["GarageArea"], 
#                               data["GarageFinish"],data["KitchenQual"], data["CentralAir"], data["isu_dist"],
#                               data["airport_dist"] , data["downtown_dist"], data["Neighborhood"]]).T

# # took out overall qual, BsmtFinSF1, LotArea, GarageCars for GarageArea, YearBuilt

In [ ]:
R_data = pd.DataFrame(data = [data["SalePrice"], data["1stFlrSF"], data["2ndFlrSF"], data["Fireplaces"], 
                              data["TotalBsmtSF"], data["ExterQual"], data["HeatingQC"],
                              data["HalfBath"], data["BsmtQual"], data["YearBuilt"], data["GarageArea"], 
                              data["GarageFinish"],data["KitchenQual"], data["CentralAir"], data["isu_dist"],
                              data["airport_dist"] , data["downtown_dist"], data["Neighborhood"]]).T

# took out overall qual, BsmtFinSF1, LotArea, GarageCars for GarageArea, YearBuilt

In [ ]:
R_data.columns

In [ ]:
pd.set_option('display.max_columns', None)
R_data.head()

In [ ]:
categorical_features = list(R_data.dtypes[R_data.dtypes == object].index)
print(len(categorical_features))
print(categorical_features)

In [ ]:
numerical_features = list(R_data.dtypes[R_data.dtypes != object].index)
print(len(numerical_features))
print(numerical_features)

In [ ]:
float_columns = R_data.loc[:, R_data.columns != "Neighborhood"]
float_columns

In [ ]:
# for i in float_columns:
#     for j in range(data.shape[0]):
#         R_data[i] = float(R_data[i][j])

In [ ]:
categorical_features = list(R_data.dtypes[R_data.dtypes == object].index)
print(len(categorical_features))
print(categorical_features)

In [ ]:
numerical_features = list(R_data.dtypes[R_data.dtypes != object].index)
print(len(numerical_features))
print(numerical_features)

In [ ]:
type(R_data[numerical_features])

In [ ]:
R_data.info()

In [ ]:
sum(pd.isnull(R_data).any())

In [ ]:
MLR_data = R_data.copy()
MLR_data['RemodFrom2011'] = 2011 - MLR_data['YearRemodAdd']
MLR_data = MLR_data.drop(['YearRemodAdd'], axis = 1)
# MLR_data.head(20)

In [ ]:
# MLR_data = R_data.copy()
# MLR_data['RemodFrom2011'] = 2011 - MLR_data['YearBuilt']
# MLR_data = MLR_data.drop(['YearBuilt'], axis = 1)
# # MLR_data.head(20)

In [ ]:
for i in neighborhoods[:3]:
    predictors         = MLR_data[MLR_data["Neighborhood"] == i]
    predictors         = predictors.drop(["Neighborhood", "SalePrice"], axis = 1)
    print(predictors.shape)

In [ ]:
lm = LinearRegression()

model_fit = {}
model_coef = {}
model_intercept = {}
model_score = {}
residuals = {}

for i in neighborhoods:
    predictors         = MLR_data[MLR_data["Neighborhood"] == i].reset_index()
    target             = predictors["SalePrice"]
    predictors         = predictors.drop(["Neighborhood", "SalePrice", "index"], axis = 1)
    model_fit[i]       = lm.fit(predictors, target)
    model_coef[i]      = model_fit[i].coef_
    model_intercept[i] = model_fit[i].intercept_
    model_score[i]     = lm.score(predictors, target)
    residuals[i]       = target - pd.Series(lm.predict(predictors))
    
    

In [ ]:
residuals['NoRidge'].shape #112 nan in 123 rows
MLR_data[MLR_data["Neighborhood"] == 'NoRidge'].shape

In [ ]:
MLR_data.groupby('Neighborhood')['SalePrice'].count().sort_values(ascending = False)

In [ ]:
# residuals 
plt_residuals = pd.DataFrame(residuals)
px.histogram(plt_residuals, x = plt_residuals.columns, nbins = 80)

In [ ]:
# R^2
model_score
['OldTown', 'NWAmes', 'CollgCr', 'NridgHt', 'Somerst', 'Gilbert']

In [ ]:
intercepts = pd.Series(model_intercept, index = model_intercept.keys()).sort_values(ascending = False)
intercepts

In [ ]:
mlr_columns = ['1stFlrSF', '2ndFlrSF', 'Fireplaces', 'TotalBsmtSF',
       'ExterQual', 'HeatingQC', 'HalfBath', 'BsmtQual',
       'GarageArea', 'GarageFinish', 'KitchenQual', 'CentralAir', 'isu_dist',
       'airport_dist', 'downtown_dist', 'YrFrom2011']
coefs = pd.DataFrame(model_coef, index = mlr_columns).T
coefs.sort_values('YrFrom2011', ascending = True)



In [ ]:
try:  
    from sklearn.model_selection import train_test_split
except:  
    from sklearn.cross_validation import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(R_data[numerical_features].loc[:,R_data[numerical_features].columns != "SalePrice"], 
                                                    R_data["SalePrice"], 
                                                    test_size=0.2, 
                                                    random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression 

ols = LinearRegression()
ols.fit(X_train, y_train)
print("R^2 for train set: %f" %ols.score(X_train, y_train))

print('-'*50)

print("R^2 for test  set: %f" %ols.score(X_test, y_test))

In [ ]:
import statsmodels.api as sm 
X_add_const = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())